# SEIRX model runs for test technology and screening frequency investigation

In [ ]:
import networkx as nx
import pandas as pd
from os.path import join

from model_SEIRX import SEIRX # agent-based simulation
import analysis_functions as af # custom analysis functions

# for progress bars
from ipywidgets import IntProgress
from IPython.display import display

**Note:** The number of simulation runs per scenario is set via the ```runs``` variable below. Running 10k simulations per scenario takes approximately 12 hours on a single ~4GHz core. Therefore, if you just want to test the simulation, I advise to set ```runs``` to for example 100. This will result in noisier statistics, but the trends will remain the same. 

In [ ]:
# number of runs per scenario
runs = 10000 
# this is where simulation results will be stored
res_path = 'data' 
# file name of the results, will be appended by the number of runs
sim_name = 'simulation_results' 
# number of employees in the simulation. Note: the number of residents
# is automatically extracted from the contact network
N_employees = 18
# resident contact network
G = nx.readwrite.gpickle.read_gpickle('data/interactions_single_quarter.gpickle')
N_residents = len(G.nodes)
# maximum number of steps in a single run. A run automatically stops
# if the outbreak is contained, i.e. there are no more infected or
# exposed individuals
N_steps = 300 
# verbosity level of the simulation. Advised to be set to zero if 
# simulations are run for data creation purposes.
verbosity = 0
# resident and employee streening intervals (in days)
r_screen_range = [2, 3, 7, None]
e_screen_range = [2, 3, 7, None]
# test technologies (and test result turnover times) used in the
# different scenarios
test_types = ['same_day_antigen', 'same_day_PCR', 'same_day_LAMP', 'one_day_PCR', 'two_day_PCR']
# specifies, whether the index case will be introduced via an
# employee or a resident
index_case_modes = ['single_employee', 'single_resident']

# progress bar
f = IntProgress(min=0, max=runs * len(r_screen_range) \
                * len(e_screen_range) * len(test_types) * len(index_case_modes)) # instantiate the bar
display(f)
c=0 # counter for progress bar

scan_results = pd.DataFrame()

for ttype in test_types:
    for index_case_mode in index_case_modes:
        for r_screen_interval in r_screen_range:
            for e_screen_interval in e_screen_range:
                # results of one ensemble, i.e. results of all runs
                # with the same parameters
                ensemble_results = pd.DataFrame()
                for r in range(runs):
                    f.value = c # update the progress bar
                    c += 1
                    
                    # instantiate model with current scenario settings
                    model = SEIRX(G, N_employees, verbosity, testing=True,
                                  preventive_screening_test_type = ttype,
                                  diagnostic_test_type = 'one_day_PCR',
                                  screening_interval_residents = r_screen_interval,
                                  screening_interval_employees = e_screen_interval,
                                  index_case_mode = index_case_mode,
                                  follow_up_testing_interval = None)
                    # run the model, end run if the outbreak is over
                    for i in range(N_steps):
                        model.step()
                        if len([a for a in model.schedule.agents if \
                            (a.exposed == True or a.infectious == True)]) == 0:
                            break
    
                    # collect the statistics of the single run
                    R0, _ = af.calculate_finite_size_R0(model)
                    infected_employees, infected_residents = af.count_infected(model)
                    data = model.datacollector.get_model_vars_dataframe()
                    num_resident_screens = data['screen_residents'].sum()
                    num_employee_screens = data['screen_employees'].sum()
                    number_of_diagnostic_tests = data['number_of_diagnostic_tests'].max()
                    number_of_preventive_screening_tests = data['number_of_preventive_screening_tests'].max()
                    transmissions = sum([a.transmissions for a in model.schedule.agents])
                    pending_test_infections = data['pending_test_infections'].max()
                    undetected_infections = data['undetected_infections'].max()
                    predetected_infections = data['predetected_infections'].max()
                    duration = len(data)
                    
                    # add run results to the ensemble results
                    ensemble_results = ensemble_results.append({'run':r, 
                                              'R0':R0,
                                              'infected_residents':infected_residents,
                                              'infected_employees':infected_employees,
                                              'N_resident_screens':num_resident_screens,
                                              'N_employee_screens':num_employee_screens,
                                              'N_diagnostic_tests':number_of_diagnostic_tests,
                                              'N_preventive_tests':number_of_preventive_screening_tests,
                                              'transmissions':transmissions,
                                              'pending_test_infections':pending_test_infections,
                                              'undetected_infections':undetected_infections,
                                              'predetected_infections':predetected_infections,
                                              'duration':duration},
                                            ignore_index=True)
                    
                # add ensemble statistics to the overall results
                row = {'test_type':ttype,
                       'index_case_mode':index_case_mode,
                       'resident_screen_interval':r_screen_interval,
                       'employee_screen_interval':e_screen_interval}
                
                row.update(af.get_statistics(ensemble_results, 'R0'))
                row.update(af.get_statistics(ensemble_results, 'infected_residents'))
                row.update(af.get_statistics(ensemble_results, 'infected_employees'))
                row.update(af.get_statistics(ensemble_results, 'N_resident_screens'))
                row.update(af.get_statistics(ensemble_results, 'N_employee_screens'))
                row.update(af.get_statistics(ensemble_results, 'N_diagnostic_tests'))
                row.update(af.get_statistics(ensemble_results, 'N_preventive_tests'))
                row.update(af.get_statistics(ensemble_results, 'transmissions'))
                row.update(af.get_statistics(ensemble_results, 'pending_test_infections'))
                row.update(af.get_statistics(ensemble_results, 'undetected_infections'))
                row.update(af.get_statistics(ensemble_results, 'predetected_infections'))
                row.update(af.get_statistics(ensemble_results, 'duration'))

                scan_results = scan_results.append(row, ignore_index=True)

# save results to disk
scan_results.to_csv(join(res_path,'{}_N{}.csv'.format(sim_name, runs)), index=False)